## Calculating likelihoods

In [1]:
import matplotlib.pyplot as plt
import corner as corner
import pandas as pd
import numpy as np
import h5py
import torch

from populations.bbh_models import get_models
import populations.bbh_models as read_models
from populations.utils.flow import NFlow
from populations.Flowsclass_dev import FlowModel
from populations import gw_obs

glasflow is using its own internal version of nflows


PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


In [2]:
#reading the pop synth file functions
def get_model_keys(path):
    alpha_val = '10'
    all_models = []
    models = []
    def find_submodels(name, obj):
        if isinstance(obj, h5py.Dataset):
            all_models.append(name.rsplit('/', 1)[0])
            
    f = h5py.File(path, 'r')
    f.visititems(find_submodels)
    # get all unique models
    all_models = sorted(list(set(all_models)))
    f.close()

    # use only models with given alpha value
    for model in all_models:
        if 'alpha' in model:
            if 'alpha'+alpha_val in model:
                models.append('/'+model)
        else:
            models.append('/' + model)
    return(np.split(np.array(models), 5))

def get_model_keys_CE(path):
    all_models = []
    models = []
    def find_submodels(name, obj):
        if isinstance(obj, h5py.Dataset):
            all_models.append(name.rsplit('/', 1)[0])
            
    f = h5py.File(path, 'r')
    f.visititems(find_submodels)
    # get all unique models
    all_models = sorted(list(set(all_models)))
    return(np.split(np.array(models), 4))

def read_hdf5(path, all_alpha=False):
    if all_alpha:
        popsynth_outputs = {}
        models = np.asarray(get_model_keys_CE(path))
        for i in range(models.shape[0]):
            for j in range(models.shape[1]):
                popsynth_outputs[i,j]=pd.read_hdf(path, key=models[i,j])
    else:
        popsynth_outputs = {}
        models = np.asarray(get_model_keys(path))
        for i in range(models.shape[0]):
            for j in range(models.shape[1]):
                popsynth_outputs[i,j]=pd.read_hdf(path, key=models[i,j])

    return(popsynth_outputs)

In [3]:
params = ['mchirp','q', 'chieff', 'z']

'obs = np.random.rand(100,46,4)\nobs[:,:,0] *=param_bounds[0,1]\nobs[:,:,1] *=param_bounds[1,1]\nobs[:,:,2] -= 0.5\nobs[:,:,2] *=param_bounds[2,1]*2\nobs[:,:,3] *=param_bounds[3,1]'

In [27]:
file_path='/Users/stormcolloms/Documents/PhD/Project_work/OneChannel_Flows/models_reduced.hdf5'
gw_path = '/Users/stormcolloms/Documents/PhD/Project_work/AMAZE_model_selection/gw_events'
observations, obsdata, p_theta, events = gw_obs.generate_observations(params, gw_path, \
                                            100, 'posteriors', None)

popsynth_outputs = read_hdf5(file_path, True)
flow = FlowModel.from_samples('CE', popsynth_outputs, params)
flow.load_model('/Users/stormcolloms/Documents/PhD/Project_work/AMAZE_model_selection/flow_models/', 'CE')

In [53]:
observations, obsdata, p_theta, events = gw_obs.generate_observations(params, gw_path, \
                                            10000, 'posteriors', None)


In [54]:
likelihoods = flow(obsdata,[3,1])

[[8.33316706e-14 6.74989417e-15 1.40664918e-13 ... 3.05537262e-11
  8.97486126e-15 7.74752441e-14]
 [3.23475913e-29 1.46733039e-17 1.80772845e-16 ... 2.73923155e-18
  2.22274049e-14 5.38213679e-13]
 [3.98253396e-05 3.65201086e-05 1.20607823e-04 ... 2.92698824e-05
  4.12492082e-05 9.04306853e-06]
 ...
 [3.02265789e-07 1.19038316e-06 2.99648968e-06 ... 5.78707443e-07
  2.97906854e-06 1.30976866e-06]
 [1.63152383e-13 1.47788601e-13 6.22116247e-12 ... 2.62943348e-11
  5.51899759e-12 4.59400747e-13]
 [8.04975357e-12 7.35457736e-23 1.25554974e-11 ... 4.32804573e-12
  5.73459292e-12 1.06214404e-11]]


In [55]:
print(likelihoods)

[4.07910525e+00 3.69565218e-01 7.71533639e-03 1.36051353e-01
 1.90474706e+02 4.78260870e-01 8.99927727e-10 2.13956522e+02
 1.99978315e+02 9.80728273e+02 1.84347904e+01 1.58913043e+01
 6.22655603e-01 2.55290761e-02 3.93231552e-01 1.38590597e-12
 4.56396097e-01 2.81134151e+00 1.34475989e-02 6.47550878e-11
 4.90044719e-10 3.52906395e-08 4.41026327e-01 8.27332347e-05
 1.77718307e+01 2.60884192e-01 5.06505807e+01 1.06847871e+02
 6.12791618e+01 3.85217796e+01 5.74991096e-02 4.95652183e+00
 1.28919210e-01 2.37869368e-09 5.86308940e-09 2.02520414e-01
 1.84717283e-01 1.76326087e+02 1.63260823e-05 2.17391304e+02
 2.81224248e+02 2.52174028e+00 3.13790051e-02 8.37177008e-04
 2.42412306e-09 1.12062876e-09]
